In [1]:
#setup

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
import sys
import os

In [2]:
os. getcwd() 

'G:\\My Drive\\PhD Program\\4_Spring 2022\\Machine Learning for Cities\\machine-learning-final\\code'

In [5]:
root= r"G:\\My Drive\\PhD Program\\4_Spring 2022\\Machine Learning for Cities\\machine-learning-final"
inp= os.path.join(root, "data", "3_clean")

In [6]:
#from pydrive.auth import GoogleAuth
#
#gauth = GoogleAuth()
#gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [7]:
filepath = "G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project/Data/3_clean_data"

In [8]:

csv_path="G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project/Data/3_clean_data/analysis_bbl.csv"
df = pd.read_csv(csv_path, low_memory=False).drop("Unnamed: 0", axis = 1)


In [9]:
print(df.columns)

Index(['borough', 'bbl', 'cd', 'tract2010', 'zipcode', 'address', 'bldgclass',
       'landuse', 'ownertype', 'ownername',
       ...
       'pctnhasian', 'pctnhhawainpi', 'pctnhother', 'pctnhtwoplus',
       'pctownerocch', 'pctrentocch', 'crowding', 'crowdingrenter',
       'medianhhincome_imp', 'geofips_recode'],
      dtype='object', length=157)


In [10]:
df= pd.get_dummies(df, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [14]:
df.head()

,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,lotarea,...,pctrentocch,crowding,crowdingrenter,medianhhincome_imp,geofips_recode,borough_BK,borough_BX,borough_MN,borough_QN,borough_SI
0,1.012090e+09,107.0,181.0,10025.0,7 WEST 95 STREET,C2,2.0,NaN,ROBERT & ANNE RABBINO,2013.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
1,1.012090e+09,107.0,181.0,10025.0,5 WEST 95 STREET,C5,2.0,NaN,"5 WEST 95TH STREET,",2517.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
2,1.012090e+09,107.0,181.0,10025.0,12 WEST 96 STREET,D4,3.0,NaN,12 W 96TH STREET CORP,10570.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
3,1.012090e+09,107.0,181.0,10025.0,44 WEST 96 STREET,D4,3.0,NaN,44 OWNERS CORP,3926.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
4,1.012090e+09,107.0,181.0,10025.0,46 WEST 96 STREET,D4,3.0,NaN,46 W 96TH ST OWNERS CORP,5035.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0


In [15]:
len(df. index) 

765085

In [16]:
#print(df.nunique())
print(df['vacate_ind'].value_counts())

0.0    762424
1.0      2604
Name: vacate_ind, dtype: int64


In [17]:
X = df.loc[:, ["bldgclass", "landuse", "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
              "assesstot", "exempttot", "yearbuilt", 'yearalter1', 'yearalter2', "holc_AB", "holc_CD", "holc_D", "viol_count", "AL", "NB",
              "average_energy_usage", "average_energy_usage_mi"]+borocols].fillna(0)

y = df.loc[:, "vacate_ind"]

In [19]:
#data1_X = data1.iloc[:,1:] # tip, distance, speed, and number of passengers
#data1_y = data1.iloc[:,0] # binary output: 1 if in Manhattan, 0 if outside
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayesDiscrete(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
        nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
    return nbc

def testNaiveBayesDiscrete(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            if thevalue not in nbc[j+'_1']:
                y_pred[i] = y_pred[i]*1E-3
            if thevalue not in nbc[j+'_0']:
                y_pred[i] = y_pred[i]*1E3
            if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [ ]:
naive_bayes_classifier = trainNaiveBayesDiscrete(X_train,y_train)
for i,j in naive_bayes_classifier.items():
    print(i)
    print(j)
    print()
y_pred_train = testNaiveBayesDiscrete(X_train,naive_bayes_classifier)
y_pred_test = testNaiveBayesDiscrete(X_test,naive_bayes_classifier)

prior
0.0033585002997727305

bldgclass_1
C1    0.242865
D1    0.201868
C7    0.134406
D7    0.071095
C0    0.061754
C2    0.050856
D4    0.031136
C3    0.026985
D3    0.025947
C4    0.019720
D6    0.018682
S9    0.014530
RM    0.012974
C5    0.010898
S5    0.009860
C6    0.007265
D9    0.006746
S4    0.006227
B2    0.005189
D5    0.005189
S3    0.004670
S2    0.004152
K4    0.004152
R4    0.004152
B9    0.003114
C9    0.003114
R1    0.002076
R3    0.001557
B3    0.001557
B1    0.001557
A9    0.001038
D8    0.001038
RR    0.000519
D0    0.000519
RD    0.000519
A2    0.000519
R2    0.000519
H6    0.000519
A5    0.000519
Name: bldgclass, dtype: float64

bldgclass_0
A1    0.161449
A5    0.136132
B1    0.112087
B2    0.103809
C0    0.095329
B3    0.079809
A2    0.058978
B9    0.034246
A9    0.025633
C3    0.021149
C1    0.019992
S2    0.019119
C2    0.017552
A0    0.011414
A3    0.010283
C7    0.009777
K4    0.008235
S1    0.007943
D1    0.006219
A4    0.005788
S9    0.005547
RM    0.004685

In [ ]:

# measure accuracy for the binary prediction task
print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

# measure accuracy of the predicted probabilities
print('Log-likelihood (train):',sum(np.log(y_pred_train*y_train+(1-y_pred_train)*(1-y_train))))
print('Log-likelihood (test):',sum(np.log(y_pred_test*y_test+(1-y_pred_test)*(1-y_test))))